# Meshing Tutorial

This tutorial is a guide on how properly mesh geometries to generate good panel method results. The power behind panel methods is the ability to capture flow behavior over **arbitrary** geometries. As a result, it is imperative that mesh refinement is incorporated in areas of high curvature and sharp corners.

```{note}
This tutorial uses [OpenVSP](https://openvsp.org/) to generate both the aircraft geometry and mesh. Users can use and combine various software tools for generating geometries ([Engineering Sketch Pad](https://acdl.mit.edu/ESP/ESPreadme.txt)) and meshes ([GMSH](https://gmsh.info/), [NetGen](https://ngsolve.org/)) as long as the standard mesh metrics and the file format are maintained.
```

This tutorial will focus on generating triangulated surface meshes, as the VortexAD panel method currently only supports this mesh type. We will illustrate this example on the ONERAM6 wing. Specs on this geometry can be found online (insert sources); we have included the OpenVSP file in the repository as well. This tutorial is focused on **mesh generation**, so we will leave out details related to geometry generation; please refer to the OpenVSP documentation for tips on generating geometries. Here we show the surface representation of the ONERA M6 geometry in OpenVSP, which is the starting point for generating a mesh.

<!-- ![ONERAM6 Geometry](images/ONERAM6_geom.png "ONERAM6 Geometry") -->

```{figure} images/ONERAM6_geom.png
---
width: 600px
name: oneram6-geometry
align: center
---
ONERAM6 Geometry in OpenVSP
```

## Mesh Control

OpenVSP has built-in mesh control capabilities that allow us to visualize the mesh during the geometry generation step. We toggle the "Hidden" surface texture to look at the mesh tessellation. The geometry is shown below.

<!-- ![ONERAM6 Grid](images/ONERAM6_geom_grid.png "ONERAM6 Grid") -->

```{figure} images/ONERAM6_geom_grid.png
---
width: 600px
name: oneram6-mesh
align: center
---
Default mesh of the ONERAM6 geometry with custom tessellation and spacing.
```

The current geometry has a certain tessellation in the "U" and "W" direction; these correspond to the grid spacing in the spanwise and airfoil cross-section, respectively. The spanwise tessellation is measured from the root to tip, and the airfoil tessellation represents the spacing from the trailing edge to the trailing edge, wrapping around the airfoil. In the OpenVSP `Geom Browser`, we can select our `WingGeom` to see the exact grid tessellation; in this instance, U is 16 and W is 45. 

As previously mentioned, mesh refinement for panel methods is key to obtain accurate solutions. Mesh refinement typically targets areas of high curvature, like the leading edge, or sharp corners, like the trailing edge and wing tips. This can be controlled through the LE/TE clustering options in the `Plan` tab and the root/tip clustering options in the `Sect` tab. The geometry above uses LE/TE clustering of 0.25/0.5 and root/tip clustering of 0.5/0.15. Below is a geometry that uses a default of 1 for each.

<!-- ![ONERAM6 Bad Grid](images/ONERAM6_geom_grid_bad.png "ONERAM6 Bad Grid") -->

```{figure} images/ONERAM6_geom_grid_bad.png
---
width: 600px
name: oneram6-mesh-bad
align: center
---
ONERAM6 wing mesh with linear spacing at the leading & trailing edges along with the wingtips - what we consider a "bad" mesh.
```

We see that the spacing of the overall grid is very uniform. This poses a problem in the areas of high curvature and sharpness, as the grid is not sufficiently modeling the curvature of the actual airfoil. We can look at the half-wing representation to get a better idea.

<!-- ```{image} images/ONERAM6_geom_grid_bad_half.png
:width: 200px
:align: center
``` -->

```{figure} images/ONERAM6_geom_grid_bad_half.png
---
width: 500px
name: oneram6-mesh-bad-half
align: center
---
A half-mesh of the ONERAM6 wing highlighting the poor representation of the leading edge curvature with no clustering.
```

We see that the leading edge curvature is being approximated by one single panel. Although the grid captures the rest of the airfoil shape very well, this inaccuracy at the leading edge severely overestimates the pressure at the leading edge, leading to issues like negative drag. 

## Mesh Generation

```{note}
To reiterate the point above, the VortexAD panel method only handles **closed**, **triangulated** meshes. Ongoing work is being done to include quad meshes and a mix of quads + triangles. Open grids will likely not be supported.
```

We will return to the mesh generation step using a "good" tessellation. Once we have generated a geometry with a sufficient tessellation, we can export to file. Navigate to the toolbar at the top of the OpenVSP window; under `File`, click `Export`. The window shown below should appear.

```{figure} images/OpenVSP_export_window.png
---
height: 600px
name: openvsp-export-window
align: center
---
OpenVSP export window
```

Select the `Stereolith (.stl)` option. Additional options will appear on the screen; these can be ignored. Select `OK`, and then choose a location to save the mesh file. Make sure to identify the full file path, as this will be needed to import the mesh when running the panel code. Once the mesh is saved to the `.stl` file, you should see the triangulated mesh appear in the OpenVSP window like below (using the "Hidden" surface texture to better identify the elements):

```{figure} images/ONERAM6_grid_stl.png
---
height: 250px
name: openvsp-export-window
align: center
---
ONERAM6 wing mesh expored in the `.stl' file format
```

``` {note}
Under the hood, **VortexAD** utilizes [**meshio**](https://pypi.org/project/meshio/) to extract mesh information; **meshio** supports many different file types, which are listed in the documentation. For now, only the `.stl` format has been tested, as it is a fairly commonly used file format for representing 3D surfaces and geometries. Other mesh file types may be explored in the future (such as `.msh` and `.dat`).
```






<!-- To test the mesh, we can also run the OpenVSP panel method through VSPAero. 
- may include errors like not water tight
- gives information about number of nodes, triangles, edges -->